# Rand 2011 Cooperation Study

This notebook outlines how to recreate the analysis of the Rand et al. 2011 study **"Dynamic social networks promote cooperation in experiments with humans"** [Link to Paper](http://humannaturelab.net/wp-content/uploads/2014/10/126-Dynamic-Social-Networks-Promote-Cooperation-in-Experiments-with-Humans.pdf "Full PDF")

This outlines the steps to re-create the analysis using the publicly available data published in the paper.  This requires either a local or remote copy of Bedrock with the following Opals installed:

* [Spreadsheet](https://github.com/Bedrock-py/opal-dataloader-ingest-spreadsheet)
* [logit2](https://github.com/Bedrock-py/opal-analytics-logit2)
* [select-from-dataframe](https://github.com/Bedrock-py/opal-analytics-select-from-dataframe)
* [summarize](https://github.com/Bedrock-py/opal-analytics-summarize)

This notebook also requires that bedrock-core be installed locally into the python kernel running this notebook.  This can be installed via command line using:

`pip install git+https://github.com/Bedrock-py/bedrock-core.git`

The other requirements to run this notebook are:

* [`pandas`]()

## Step 1: Check Environment

First check that Bedrock is installed locally.  If the following cell does not run without error, check the install procedure above and try again.  Also, ensure that the kernel selected is the same as the kernel where bedrock-core is installed

In [1]:
from bedrock.client.client import BedrockAPI

### Test Connection to Bedrock Server

This code assumes a local bedrock is hosted at localhost on port 81.  Change the `SERVER` variable to match your server's URL and port.

In [2]:
import requests
import pandas
import pprint
SERVER = "http://localhost:81/"
api = BedrockAPI(SERVER)

### Check for Spreadsheet Opal

The following code block checks the Bedrock server for the Spreadsheet Opal.  This Opal is used to load .csv, .xls, and other such files into a Bedrock matrix format.  The code below calls the Bedrock `/dataloaders/ingest` endpoint to check if the `opals.spreadsheet.Spreadsheet.Spreadsheet` opal is installed.

If the code below shows the Opal is not installed, there are two options:
1. If you are running a local Bedrock or are the administrator of the Bedrock server, install the Spreadsheet Opal with pip on the server [Spreadsheet](https://github.com/Bedrock-py/opal-dataloader-ingest-spreadsheet)
2. If you are not administrator of the Bedrock server, e-mail the Bedrock administrator requesting the Opal be installed

In [3]:
resp = api.ingest("opals.spreadsheet.Spreadsheet.Spreadsheet")
if resp.json():
    print("Spreadsheet Opal Installed!")
else:
    print("Spreadsheet Opal Not Installed!")

Spreadsheet Opal Installed!


### Check for logit2 Opal

The following code block checks the Bedrock server for the logit2 Opal. 

If the code below shows the Opal is not installed, there are two options:
1. If you are running a local Bedrock or are the administrator of the Bedrock server, install the logit2 Opal with pip on the server [logit2](https://github.com/Bedrock-py/opal-analytics-logit2)
2. If you are not administrator of the Bedrock server, e-mail the Bedrock administrator requesting the Opal be installed

In [4]:
resp = api.analytic('opals.logit2.Logit2.Logit2')
if resp.json():
    print("Logit2 Opal Installed!")
else:
    print("Logit2 Opal Not Installed!")

Logit2 Opal Installed!


### Check for select-from-dataframe Opal

The following code block checks the Bedrock server for the select-from-dataframe Opal. This allows you to filter by row and reduce the columns in a dataframe loaded by the server. 

If the code below shows the Opal is not installed, there are two options:
1. If you are running a local Bedrock or are the administrator of the Bedrock server, install the select-from-datafram Opal with pip on the server [select-from-dataframe](https://github.com/Bedrock-py/opal-analytics-select-from-dataframe)
2. If you are not administrator of the Bedrock server, e-mail the Bedrock administrator requesting the Opal be installed

In [5]:
resp = api.analytic('opals.select-from-dataframe.SelectByCondition.SelectByCondition')
if resp.json():
    print("Select-from-dataframe Opal Installed!")
else:
    print("Select-from-dataframe Opal Not Installed!")

Select-from-dataframe Opal Installed!


### Check for summarize Opal

The following code block checks the Bedrock server for the summarize Opal. This allows you to summarize a matrix with an optional groupby clause.

If the code below shows the Opal is not installed, there are two options:
1. If you are running a local Bedrock or are the administrator of the Bedrock server, install the summarize with pip on the server [summarize](https://github.com/Bedrock-py/opal-analytics-summarize)
2. If you are not administrator of the Bedrock server, e-mail the Bedrock administrator requesting the Opal be installed

In [6]:
resp = api.analytic('opals.summarize.Summarize.Summarize')
if resp.json():
    print("Summarize Opal Installed!")
else:
    print("Summarize Opal Not Installed!")

Summarize Opal Installed!


## Step 2: Upload Data to Bedrock and Create Matrix

Now that everything is installed, begin the workflow by uploading the csv data and creating a matrix.  To understand this fully, it is useful to understand how a data loading workflow occurs in Bedrock.

1. Create a datasource that points to the original source file
2. Generate a matrix from the data source (filters can be applied during this step to pre-filter the data source on load
3. Analytics work on the generated matrix

** Note: Each time a matrix is generated from a data source it will create a new copy with a new UUID to represent that matrix **

### Check for csv file locally

The following code opens the file and prints out the first part.  The file must be a csv file with a header that has labels for each column.  The file is comma delimited csv.

In [22]:
filepath = 'cooperation_exp1.csv'
datafile = pandas.read_csv(filepath)
datafile.head(10)

,entry,session,condition,pid,action,previous_decision,round,num_neighbors,group_size,fluid_dummy
0,1,1,Static,143,0,NaN,1,5,32,0
1,2,1,Static,144,0,NaN,1,11,32,0
2,3,1,Static,113,0,NaN,1,3,32,0
3,4,1,Static,105,0,NaN,1,7,32,0
4,5,1,Static,107,1,NaN,1,6,32,0
5,6,1,Static,111,1,NaN,1,2,32,0
6,7,1,Static,121,1,NaN,1,1,32,0
7,8,1,Static,112,0,NaN,1,4,32,0
8,9,1,Static,109,0,NaN,1,5,32,0
9,10,1,Static,137,1,NaN,1,4,32,0


### Now Upload the source file to the Bedrock Server

This code block uses the Spreadsheet ingest module to upload the source file to Bedrock.  ** Note: This simply copies the file to the server, but does not create a Bedrock Matrix format **

If the following fails to upload. Check that the csv file is in the correct comma delimited format with headers.

In [23]:
ingest_id = 'opals.spreadsheet.Spreadsheet.Spreadsheet'
resp = api.put_source('cooperation_exp1', ingest_id, 'default', {'file': open(filepath, "rb")})

if resp.status_code == 201:
    source_id = resp.json()['src_id']
    print('Source {0} successfully uploaded'.format(filepath))
else:
    try:
        print("Error in Upload: {}".format(resp.json()['msg']))
    except Exception:
        pass
    
    try:
        source_id = resp.json()['src_id']
        print("Using existing source.  If this is not the desired behavior, upload with a different name.")
    except Exception:
        print("No existing source id provided")

Source cooperation_exp1.csv successfully uploaded


### Check available data sources for the CSV file

Call the Bedrock sources list to see available data sources.  Note, that the `Rand2011` data source should now be available

In [24]:
available_sources = api.list("dataloader", "sources").json()
s = next(filter(lambda source: source['src_id'] == source_id, available_sources),'None')
if s != 'None':
    pp = pprint.PrettyPrinter()
    pp.pprint(s)
else:
    print("Could not find source")

{'count': 0,
 'created': '2017-09-15 15:10:21.065777',
 'filepath': '/opt/bedrock/dataloader/data/fd863481751a4f1b8ea0e6ac651e2ac1/source/cooperation_exp1.csv',
 'group_name': 'default',
 'host': '172.17.0.2',
 'ingest_id': 'opals.spreadsheet.Spreadsheet.Spreadsheet',
 'matrices': [],
 'name': 'cooperation_exp1',
 'rootdir': '/opt/bedrock/dataloader/data/fd863481751a4f1b8ea0e6ac651e2ac1/',
 'src_id': 'fd863481751a4f1b8ea0e6ac651e2ac1',
 'src_type': 'file',
 'status': []}


### Create a Bedrock Matrix from the CSV Source

In order to use the data, the data source must be converted to a Bedrock matrix.  The following code steps through that process.  Here we are doing a simple transform of csv to matrix.  There are options to apply filters (like renaming columns, excluding colum

In [25]:
resp = api.create_matrix(source_id, 'cooperation_exp1_mtx')
mtx = resp[0]
matrix_id = mtx['id']
print(mtx)
resp

{'id': '54c2c347c3624745bacfec582a2997dc', 'created': '2017-09-15 15:10:25.714282', 'mat_type': 'csv', 'outputs': ['features_original.txt', 'features.txt', 'matrix.csv'], 'name': 'cooperation_exp1_mtx', 'filters': {'action': {}, 'num_neighbors': {}, 'round': {}, 'pid': {}, 'group_size': {}, 'session': {}, 'entry': {}, 'condition': {}, 'fluid_dummy': {}, 'previous_decision': {}}, 'rootdir': '/opt/bedrock/dataloader/data/fd863481751a4f1b8ea0e6ac651e2ac1/54c2c347c3624745bacfec582a2997dc/', 'src_id': 'fd863481751a4f1b8ea0e6ac651e2ac1'}


[{'created': '2017-09-15 15:10:25.714282',
  'filters': {'action': {},
   'condition': {},
   'entry': {},
   'fluid_dummy': {},
   'group_size': {},
   'num_neighbors': {},
   'pid': {},
   'previous_decision': {},
   'round': {},
   'session': {}},
  'id': '54c2c347c3624745bacfec582a2997dc',
  'mat_type': 'csv',
  'name': 'cooperation_exp1_mtx',
  'outputs': ['features_original.txt', 'features.txt', 'matrix.csv'],
  'rootdir': '/opt/bedrock/dataloader/data/fd863481751a4f1b8ea0e6ac651e2ac1/54c2c347c3624745bacfec582a2997dc/',
  'src_id': 'fd863481751a4f1b8ea0e6ac651e2ac1'}]

### Look at basic statistics on the source data

Here we can see that Bedrock has computed some basic statistics on the source data.

#### For numeric data

The quartiles, max, mean, min, and standard deviation are provided

#### For non-numeric data

The label values and counts for each label are provided.

#### For both types

The proposed tags and data type that Bedrock is suggesting are provided

In [26]:
analytic_id = "opals.summarize.Summarize.Summarize"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = []

summary_mtx = api.run_analytic(analytic_id, mtx, 'cooperation_exp1_summary', input_data=inputData, parameter_data=paramsData)
output = api.download_results_matrix(matrix_id, summary_mtx['id'], 'matrix.csv')
output

,0,1,2,3,4,5,6,7,8,9
0,NaN,entry,session,pid,action,previous_decision,round,num_neighbors,group_size,fluid_dummy
1,count,3876.0,3876.0,3876.0,3876.0,3091.0,3876.0,3876.0,3876.0,3876.0
2,mean,1938.5,21.8268833849,2196.08642931,0.532507739938,0.536072468457,3.86300309598,5.23942208462,20.1109391125,0.266769865841
3,std,1119.04914995,15.4079731245,1538.91902422,0.499006503227,0.49877776821,2.60690174006,3.30793050227,6.21864617943,0.442328139389
4,min,1.0,1.0,102.0,0.0,0.0,1.0,0.0,10.0,0.0
5,25%,969.75,9.0,918.0,0.0,0.0,2.0,3.0,15.0,0.0
6,50%,1938.5,19.0,1915.0,1.0,1.0,3.0,5.0,19.0,0.0
7,75%,2907.25,34.0,3407.25,1.0,1.0,5.0,7.0,25.0,1.0
8,max,3876.0,52.0,5240.0,1.0,1.0,11.0,18.0,33.0,1.0


## Step 3: Create Filtered Matrices on each Game Condition

In [33]:
analytic_id = "opals.select-from-dataframe.SelectByCondition.SelectByCondition"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
    {"attrname":"colname","value":"condition"},
    {"attrname":"comparator","value":"=="},
    {"attrname":"value","value":"Static"}
]

cooperation_exp1_static = api.run_analytic(analytic_id, mtx, 'cooperation_exp1_static', input_data=inputData, parameter_data=paramsData)

cooperation_exp1_static

{'analytic_id': 'opals.select-from-dataframe.SelectByCondition.SelectByCondition',
 'created': '2017-09-15 15:19:54.033339',
 'id': '031fc8522616494395c75aa6e42156ee',
 'name': 'cooperation_exp1_static',
 'outputs': ['matrix.csv', 'features.txt'],
 'parameters': [{'attrname': 'colname', 'value': 'condition'},
  {'attrname': 'comparator', 'value': '=='},
  {'attrname': 'value', 'value': 'Static'}],
 'rootdir': '/opt/bedrock/analytics/data/54c2c347c3624745bacfec582a2997dc/031fc8522616494395c75aa6e42156ee/',
 'src_id': '54c2c347c3624745bacfec582a2997dc'}

In [34]:
analytic_id = "opals.select-from-dataframe.SelectByCondition.SelectByCondition"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
    {"attrname":"colname","value":"condition"},
    {"attrname":"comparator","value":"=="},
    {"attrname":"value","value":"Random"}
]

cooperation_exp1_random = api.run_analytic(analytic_id, mtx, 'cooperation_exp1_random', input_data=inputData, parameter_data=paramsData)

cooperation_exp1_random

{'analytic_id': 'opals.select-from-dataframe.SelectByCondition.SelectByCondition',
 'created': '2017-09-15 15:20:01.278110',
 'id': '521b155e3e4b4bf2bfffe6e1ea3ed1ec',
 'name': 'cooperation_exp1_random',
 'outputs': ['matrix.csv', 'features.txt'],
 'parameters': [{'attrname': 'colname', 'value': 'condition'},
  {'attrname': 'comparator', 'value': '=='},
  {'attrname': 'value', 'value': 'Random'}],
 'rootdir': '/opt/bedrock/analytics/data/54c2c347c3624745bacfec582a2997dc/521b155e3e4b4bf2bfffe6e1ea3ed1ec/',
 'src_id': '54c2c347c3624745bacfec582a2997dc'}

In [35]:
analytic_id = "opals.select-from-dataframe.SelectByCondition.SelectByCondition"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
    {"attrname":"colname","value":"condition"},
    {"attrname":"comparator","value":"=="},
    {"attrname":"value","value":"Viscous"}
]

cooperation_exp1_viscous = api.run_analytic(analytic_id, mtx, 'cooperation_exp1_viscous', input_data=inputData, parameter_data=paramsData)

cooperation_exp1_viscous

{'analytic_id': 'opals.select-from-dataframe.SelectByCondition.SelectByCondition',
 'created': '2017-09-15 15:20:07.743073',
 'id': '4d6b17f9ebdf45f7baefbfeb4f8e54ff',
 'name': 'cooperation_exp1_viscous',
 'outputs': ['matrix.csv', 'features.txt'],
 'parameters': [{'attrname': 'colname', 'value': 'condition'},
  {'attrname': 'comparator', 'value': '=='},
  {'attrname': 'value', 'value': 'Viscous'}],
 'rootdir': '/opt/bedrock/analytics/data/54c2c347c3624745bacfec582a2997dc/4d6b17f9ebdf45f7baefbfeb4f8e54ff/',
 'src_id': '54c2c347c3624745bacfec582a2997dc'}

In [36]:
analytic_id = "opals.select-from-dataframe.SelectByCondition.SelectByCondition"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
    {"attrname":"colname","value":"condition"},
    {"attrname":"comparator","value":"=="},
    {"attrname":"value","value":"Fluid"}
]

cooperation_exp1_fluid = api.run_analytic(analytic_id, mtx, 'cooperation_exp1_fluid', input_data=inputData, parameter_data=paramsData)

cooperation_exp1_fluid

{'analytic_id': 'opals.select-from-dataframe.SelectByCondition.SelectByCondition',
 'created': '2017-09-15 15:20:14.473065',
 'id': '120b7c2659d34b238ee2bce5253a48a5',
 'name': 'cooperation_exp1_fluid',
 'outputs': ['matrix.csv', 'features.txt'],
 'parameters': [{'attrname': 'colname', 'value': 'condition'},
  {'attrname': 'comparator', 'value': '=='},
  {'attrname': 'value', 'value': 'Fluid'}],
 'rootdir': '/opt/bedrock/analytics/data/54c2c347c3624745bacfec582a2997dc/120b7c2659d34b238ee2bce5253a48a5/',
 'src_id': '54c2c347c3624745bacfec582a2997dc'}

## Hypothesis 4.1.1

In [39]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': cooperation_exp1_static,
    'features.txt': cooperation_exp1_static
}

paramsData = [
    {"attrname":"formula","value":"action ~ round"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"session,pid"}
]

hyp_4_1_1 = api.run_analytic(analytic_id, mtx, 'exp1_hypothesis_4_1_1', input_data=inputData, parameter_data=paramsData)

hyp_4_1_1

{'analytic_id': 'opals.logit2.Logit2.Logit2',
 'created': '2017-09-15 15:22:42.886299',
 'id': '52283aed365f4dae876271d321d38247',
 'name': 'exp1_hypothesis_4_1_1',
 'outputs': ['matrix.csv', 'summary.csv'],
 'parameters': [{'attrname': 'formula', 'value': 'action ~ round'},
  {'attrname': 'family', 'value': 'binomial'},
  {'attrname': 'clustered_rse', 'value': 'session,pid'}],
 'rootdir': '/opt/bedrock/analytics/data/54c2c347c3624745bacfec582a2997dc/52283aed365f4dae876271d321d38247/',
 'src_id': '54c2c347c3624745bacfec582a2997dc'}

### Visualize the output of the analysis

Here the output of the analysis is downloaded and from here can be visualized and exported

In [41]:
coef_table = api.download_results_matrix(hyp_4_1_1['src_id'], hyp_4_1_1['id'], 'matrix.csv')
coef_table

,0,1,2,3,4,5,6
0,NaN,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
1,Intercept,0.661937181574,0.195661719654,3.38306942588,0.000716805093876,0.2784472579,1.04542710525
2,round,-0.186475503388,0.0363630888409,-5.12815355714,2.92597743001e-07,-0.257745847883,-0.115205158893


In [42]:
summary_table = api.download_results_matrix(hyp_4_1_1['src_id'], hyp_4_1_1['id'], 'summary.csv')
summary_table

,0,1
0,AIC,1340.369608
1,deviance,1336.369608
2,peasron_chi2,1007.692219


# EVERYTHING BELOW HERE IS LEFTOVER FROM PREV EXAMPLE AND NOT YET READY

## Apply method to complete Rand2011 Analysis

The following cells replicate the other analysis pieces from the Rand2011 study

### Summarize decision grouped on condition and round_num

In [ ]:
analytic_id = "opals.summarize.Summarize.Summarize"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
    {"attrname":"groupby","value":"condition,round_num"},
    {"attrname":"columns","value":"decision0d1c"}
]

base_mtx = api.get_matrix_metadata('Rand2011','rand_mtx')

summary_mtx = api.run_analytic(analytic_id, base_mtx,'summarize_grouped', input_data=inputData, parameter_data=paramsData)
output = api.download_results_matrix(base_mtx['id'], summary_mtx['id'], 'matrix.csv')
output

### Compare round_num effect on decision

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': base_mtx,
    'features.txt': base_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ round_num"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, base_mtx, 'rand_logit2_step1', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table

### Consider only num_neighbors > 0

In [ ]:
analytic_id = "opals.select-from-dataframe.SelectByCondition.SelectByCondition"
inputData = {
    'matrix.csv': base_mtx,
    'features.txt': base_mtx
}

paramsData = [
    {"attrname":"colname","value":"num_neighbors"},
    {"attrname":"comparator","value":">"},
    {"attrname":"value","value":"0"}
]

filtered_mtx = api.run_analytic(analytic_id, base_mtx, 'rand_has_neighbors', input_data=inputData, parameter_data=paramsData)

#### Summarize on filtered matrix

In [ ]:
analytic_id = "opals.summarize.Summarize.Summarize"
inputData = {
    'matrix.csv': filtered_mtx,
    'features.txt': filtered_mtx
}

paramsData = [
    {"attrname":"groupby","value":"condition,round_num"},
    {"attrname":"columns","value":"decision0d1c"}
]

summary_mtx = api.run_analytic(analytic_id, filtered_mtx,'summarize_grouped', input_data=inputData, parameter_data=paramsData)
output = api.download_results_matrix(base_mtx['id'], summary_mtx['id'], 'matrix.csv')
output

#### Compare round_num effect on decision only when there are neighbors

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': filtered_mtx,
    'features.txt': filtered_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ round_num"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, filtered_mtx, 'rand_logit2_step2', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table

## Compare effect of round_num and Fluid

Look at the effect the round number an if the game is Fluid.

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': base_mtx,
    'features.txt': base_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ fluid_dummy*round_num"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, base_mtx, 'rand_logit2_step4', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table

## Condition effect on decision for Round >= 7

In [ ]:
analytic_id = "opals.select-from-dataframe.SelectByCondition.SelectByCondition"
inputData = {
    'matrix.csv': base_mtx,
    'features.txt': base_mtx
}

paramsData = [
    {"attrname":"colname","value":"round_num"},
    {"attrname":"comparator","value":">="},
    {"attrname":"value","value":"7"}
]

filtered_mtx = api.run_analytic(analytic_id, base_mtx, 'rand_round7', input_data=inputData, parameter_data=paramsData)

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': filtered_mtx,
    'features.txt': filtered_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ condition"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, filtered_mtx, 'rand_logit2_step5', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table

## Fluid Effect on decision for Round >= 7

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': filtered_mtx,
    'features.txt': filtered_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ C(fluid_dummy)"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, filtered_mtx, 'rand_logit2_step6', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table

## Relevel on Random and Compare condition effect on decision

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': base_mtx,
    'features.txt': base_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ C(condition, Treatment(reference='Random'))"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, base_mtx, 'rand_logit2_step7', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
pandas.set_option('display.max_colwidth', -1)
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table

## Relevel on Static and Compare condition effect on decision

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': base_mtx,
    'features.txt': base_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ C(condition, Treatment(reference='Static'))"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, base_mtx, 'rand_logit2_step8', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
pandas.set_option('display.max_colwidth', -1)
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table

## Relevel on Random and round_num >= 7

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': filtered_mtx,
    'features.txt': filtered_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ C(condition, Treatment(reference='Random'))"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, filtered_mtx, 'rand_logit2_step9', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table

## Relevel on Static and round_num >= 7

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': filtered_mtx,
    'features.txt': filtered_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ C(condition, Treatment(reference='Static'))"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, filtered_mtx, 'rand_logit2_step10', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table

## Subset on Fluid Condition and look at effect of num_neighbors on decision

In [ ]:
analytic_id = "opals.select-from-dataframe.SelectByCondition.SelectByCondition"
inputData = {
    'matrix.csv': base_mtx,
    'features.txt': base_mtx
}

paramsData = [
    {"attrname":"colname","value":"condition"},
    {"attrname":"comparator","value":"=="},
    {"attrname":"value","value":"Fluid"}
]

filtered_mtx = api.run_analytic(analytic_id, base_mtx, 'rand_fluid_only', input_data=inputData, parameter_data=paramsData)

In [ ]:
analytic_id = "opals.logit2.Logit2.Logit2"
inputData = {
    'matrix.csv': filtered_mtx,
    'features.txt': filtered_mtx
}

paramsData = [
    {"attrname":"formula","value":"decision0d1c ~ C(num_neighbors)"},
    {"attrname":"family","value":"binomial"},
    {"attrname":"clustered_rse","value":"sessionnum,playerid"}
]

result_mtx = api.run_analytic(analytic_id, filtered_mtx, 'rand_logit2_step11', input_data=inputData, parameter_data=paramsData)
coef_table = api.download_results_matrix(base_mtx['id'], result_mtx['id'], 'matrix.csv')
coef_table

In [ ]:
summary_table = api.download_results_matrix(result_mtx['src_id'], result_mtx['id'], 'summary.csv')
summary_table